In [ ]:
# Monitors library source files and recompiles them after most changes
import Revise

# Run the init script which will setup the JDP project if necessary
include("../src/init.jl")

using Markdown

# Import some libraries from the JDP project
using JDP.BugRefs
using JDP.Tracker
using JDP.Trackers.OpenQA    # Contains functions for dealing with the OpenQA web API
using JDP.Trackers.Bugzilla  # Functions for accessing the Bugzilla API(s)
using JDP.Repository
using JDP.Functional

trackers = load_trackers();

# Bug Tag Propagation

This partially automates copying existing bug tags from one failed test result to another.

## Find untagged test failures

First we fetch the test results from OpenQA.

In [ ]:
OpenQA.refresh_comments(job -> job.vars["BUILD"] == "158.4", "osd")

In [ ]:
# Get some job results from the openqa.suse.de (osd) OpenQA instance.
# Optional arguments (after the ';') like 'groupid' are passed to the OpenQA API
allres = Repository.fetch(OpenQA.TestResult, Vector, "osd"; refresh=false, groupid=116)

Then filter the results to only include fails from a particular build with no bug references.

In [ ]:
build = "158.4"

untagged = filter(allres) do res
    res.build == build &&
    occursin(r"failed", res.result) &&
    (isempty(res.refs) || all(rf -> rf.negated, res.refs))
end

for res in untagged
    display(res)
end

## Find existing bug references for tests

Now we search all the test results for bug tags that were added in other builds or architectures.

In [ ]:
fqn = OpenQA.get_fqn

tagdict = Dict{String, Set{JDP.BugRefs.Ref}}(fqn(res) => Set() for res in untagged)

for res in allres
    if haskey(tagdict, fqn(res)) && length(res.refs) > 0
        refs = tagdict[fqn(res)]
        
        for rf in res.refs
            # Negated bugrefs premenently stop a bugref from being used on the same test
            if rf.negated
                delete!(refs, BugRefs.Ref(rf.tracker, rf.id, false))
                push!(refs, rf)
            elseif !(BugRefs.Ref(rf.tracker, rf.id, true) in refs)
                push!(refs, rf)
            end
        end
    end
end

length(tagdict)

In [ ]:
filter(p -> !isempty(p[2]), pairs(tagdict))

Optionally we can get a summary of the bug tags it has found from Bugzilla.

> TODO: look on Progress instead for poo refs

In [ ]:
bugdict = Dict(name => [] for name in keys(tagdict))
for (k, v) in pairs(tagdict)
    for rf in v
        bug = Repository.fetch(Bugzilla.Bug, rf)
        if bug != nothing
            push!(bugdict[k], rf => bug)
        end
    end
end

bugdict

In [ ]:
mdbuf = IOBuffer()

for (k, v) in pairs(bugdict)
    if isempty(v)
        continue
    end
    
    println(mdbuf, "- ", k)
    for (rf, bug) in v
        print(mdbuf, "   * ")
        show(mdbuf, MIME("text/markdown"), rf)
        print(mdbuf, " ")
        show(mdbuf, MIME("text/markdown"), bug)
        println(mdbuf)
    end
    
end

seek(mdbuf, 0)
Markdown.parse(mdbuf)

## Manual additions and removals

If necessary some tags can be removed, although you should generally use anti-tags or modify the search algorithm.

In [ ]:
modded = []

for (t, rf) in ["fstests-btrfs-generic-521" => "bsc#1119212", 
                "fstests-btrfs-generic-263" => "bsc#1119212",
                "fstests-btrfs-generic-091" => "bsc#1119212",
                "LTP-syscalls-fanotify09" => "bsc#1110880"]
    if haskey(tagdict, t)
        delete!(tagdict[t], BugRefs.Ref(rf, trackers))
        push!(modded, tagdict[t])
    end
end

modded

While others can be added, which may be quicker than commenting directly on OpenQA or implementing some new tagging logic.

## Posting back to OpenQA

Now we can post the results back to OpenQA. First we create a dictionary of jobs which we will post the bug tags to.

In [ ]:
taggings = Dict()

for res in untagged
    for rf in tagdict[fqn(res)]
        if rf.tracker.tla != "poo"
            tags = get!(taggings, res.job.id, [])
            push!(tags, res.name => rf)
        end
    end
end
        
taggings     

In [ ]:
oqa = get_tracker(trackers, "osd")
ses = Tracker.ensure_login!(oqa)

for (jid, refs) in taggings
    mdbuf = IOBuffer()
    
    print(mdbuf, 
        "This is an automated message from [JDP](https://gitlab.suse.de/rpalethorpe/jdp)",
        "<br><br>",
        "The following bug tags have been propogated: <br>")
    for (name, rf) in refs
        print(mdbuf, name, ": ", rf, "<br>")
    end
    
    print(mdbuf, "<br>This superscedes any (Automatic takeover from t#...) messages")
    text = String(take!(mdbuf))
    
    @info "Posting comment to job $jid: \n$text"
    OpenQA.post_job_comment(ses, jid, text)
end

## Failures still missing a tag

In [ ]:
filter(kv -> isempty(kv.second), pairs(tagdict)) |> keys